## Select Assignment Number

To load the correct settings, the number for the current assignment is set. 

In [7]:
assignment_nr = 1

## Load Packages and Global Settings

In [4]:
# Local imports
from scripts.jsonify import jsonify, jsonify_resources, analyze_jsonify_results

In [1]:
# Load global settings
from scripts.settings import *

In [2]:
# Set week number
week_nr = 1

# Load week specific settings
ASSIGNMENT = ASSIGNMENTS[assignment_nr]
ASSIGNMENT_ID = ASSIGNMENT["canvas"]["assignment_id"]
QUIZ_ID = ASSIGNMENT["canvas"]["quiz_id"]
R_QUIZ_QUESTION_ID = ASSIGNMENT["canvas"]["r_quiz_question_id"]
ADV_QUIZ_QUESTION_ID = ASSIGNMENT["canvas"]["adv_quiz_question_id"]
LOCK_GRADES_DATE = ASSIGNMENT["lock_grades_date"]

## Jsonify Resources

To ensure the latest changes to rubrics, assignment, example solutions, or goals are captured, the resources are jsonified.

In [6]:
analyze_jsonify_results(jsonify_resources(week_nr, RESOURCES_PATH))

questions: 48 (R: 30, Radv: 8, Python: 10)
rubrics: 48 (R: 30, Radv: 8, Python: 10)
solutions: 48 (R: 30, Radv: 8, Python: 10)
goals: 48 (R: 30, Radv: 8, Python: 10)


## Download Submissions

All assignment submissions are downloaded.